In [3]:
import os
import numpy as np
import nd2
import piscis

In [5]:
icos_img_dir = "../tobias_ICOS"

In [13]:
##### Read in .nd2 files with nd2 library
loc1 = nd2.imread(os.path.join(icos_img_dir, "Myla_ICOS_slide1_well1_loc001.nd2")) # Shape is (Z, C, Y, X)
loc1 = np.transpose(loc1, (1,0,2,3)) # Transpose to (C, Z, Y, X)

In [16]:
##### Select desired channel
loc_dot_channel = loc1[:,1]

In [11]:
##### Piscis
model = piscis.Piscis(model_name='20230905')

wniu/Piscis/models/20230905:   0%|          | 0.00/29.9M [00:00<?, ?B/s]

In [22]:
spots_pred = model.predict(loc_dot_channel, threshold=0.5)

In [21]:
for i in spots_pred:
    print(len(i))

65
71
99
96
85
90
80
76
85
108
99
89
87
80
91
95
91
97
96
95
99
91
88
83
87
80
82
88
83
88
90
84
85
86
73
